In [ ]:
from typing import Dict, Any, List, Optional
from pydantic import BaseModel
import json
import logging

logger = logging.getLogger(__name__)

class DocumentMetadata(BaseModel):
    id: str
    url: str
    title: str
    properties: Dict[str, Any]

class EnhancedNotionLoader:
    def __init__(self, integration_token: Optional[str] = None):
        """Initialize the NotionDatabaseClient.
        
        Args:
            integration_token: Optional Notion API key. If not provided, will use from settings.
        """
        # You would replace this with your settings import
        from your_config import settings  # Replace with your actual settings
        
        self.integration_token = integration_token or settings.NOTION_API_KEY
        
        assert self.integration_token is not None, (
            "NOTION_API_KEY environment variable is required. Set it in your .env file."
        )
        
        # Import the notion client here to use the token
        from langchain.document_loaders.notiondb import NotionClient
        self.notion_client = NotionClient(auth=self.integration_token)
        
    def query_notion_database(self, database_id: str, query_json: Optional[str] = None) -> List[DocumentMetadata]:
        """Query a Notion database and return its results.

        Args:
            database_id: The ID of the Notion database to query.
            query_json: Optional JSON string containing query parameters.

        Returns:
            A list of DocumentMetadata objects containing the query results.
        """
        query_payload = {}
        if query_json and query_json.strip():
            try:
                query_payload = json.loads(query_json)
            except json.JSONDecodeError:
                logger.debug("Invalid JSON format for query", exc_info=True)
                return []
                
        try:
            response = self.notion_client.databases.query(
                database_id=database_id,
                **query_payload
            )
            results = response["results"]
        except Exception as e:
            logger.debug(f"Error querying Notion database: {e}", exc_info=True)
            return []
            
        return [self.__build_page_metadata(page) for page in results]
        
    def __build_page_metadata(self, page: Dict[str, Any]) -> DocumentMetadata:
        """Build a DocumentMetadata object from a Notion page dictionary.

        Args:
            page: Dictionary containing Notion page data.

        Returns:
            A DocumentMetadata object containing the processed page data.
        """
        properties = self.__flatten_properties(page.get("properties", {}))
        title = properties.pop("Name", "Untitled")
        
        if page.get("parent"):
            properties["parent"] = {
                "id": page["parent"]["database_id"],
                "url": "",
                "title": "",
                "properties": {},
            }
            
        return DocumentMetadata(
            id=page["id"],
            url=page["url"],
            title=title,
            properties=properties
        )
        
    def __flatten_properties(self, properties: Dict) -> Dict:
        """Flatten Notion properties dictionary into a simpler key-value format.

        Args:
            properties: Dictionary of Notion properties to flatten.

        Returns:
            A flattened dictionary with simplified key-value pairs.
        """
        flattened = {}
        
        for key, value in properties.items():
            prop_type = value.get("type")
            
            if prop_type == "select":
                select_value = value.get("select", {}) or {}
                flattened[key] = select_value.get("name")
            elif prop_type == "multi_select":
                flattened[key] = [
                    item.get("name") for item in value.get("multi_select", [])
                ]
            elif prop_type == "title":
                flattened[key] = "\n".join(
                    item.get("plain_text", "") for item in value.get("title", [])
                )
            elif prop_type == "rich_text":
                flattened[key] = " ".join(
                    item.get("plain_text", "") for item in value.get("rich_text", [])
                )
            elif prop_type == "number":
                flattened[key] = value.get("number")
            elif prop_type == "checkbox":
                flattened[key] = value.get("checkbox")
            elif prop_type == "date":
                date_value = value.get("date", {})
                if date_value:
                    flattened[key] = {
                        "start": date_value.get("start"),
                        "end": date_value.get("end"),
                    }
            elif prop_type == "database_id":
                flattened[key] = value.get("database_id")
            else:
                flattened[key] = value
                
        return flattened

In [2]:
from enhanced_notion_loader import EnhancedNotionLoader

# Replace with your actual token and database ID
NOTION_TOKEN = "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"
DATABASE_ID = "15bfed67a6ba8071ac03d29c75a725da"

def test_notion_integration():
    # Initialize the loader
    loader = EnhancedNotionLoader(integration_token=NOTION_TOKEN)
    
    try:
        # Query the database
        results = loader.query_notion_database(DATABASE_ID)
        
        # Print results to verify
        print(f"Successfully retrieved {len(results)} pages")
        
        if results:
            # Print the first result to verify structure
            first_page = results[0]
            print("\nFirst page details:")
            print(f"ID: {first_page.id}")
            print(f"Title: {first_page.title}")
            print(f"URL: {first_page.url}")
            print("\nProperties:")
            for key, value in first_page.properties.items():
                print(f"  {key}: {value}")
                
        return len(results) > 0
        
    except Exception as e:
        print(f"Error testing Notion integration: {e}")
        return False

if __name__ == "__main__":
    success = test_notion_integration()
    print(f"\nTest {'succeeded' if success else 'failed'}")

ModuleNotFoundError: No module named 'enhanced_notion_loader'